In [ ]:
# Colab Conversion to SSH Server Code Created by Imad El Hanafi : https://imadelhanafi.com
# Machine Learning DDQN Hive Mind programmed by Clint Wang
# Sockets programmed by Dhruv Chanana

# Check GPU status

Make surre to use : GPU runtime mode (Runtime->Change Runtime type -> python3 + GPU
)



In [ ]:
# Check nvidia and nvcc cuda compiler

!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Fri Jul 24 15:09:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Mount Google Drive for SSH Files

In [ ]:
# link to google drive

from google.colab import drive
drive.mount('/content/gdrive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
#check that Gdrive is mounted

!pwd

/content


#Setup SSH port forwarding

In [ ]:
# 2 - Download Ngrok

! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

In [ ]:
# 3 - setup Ngrok - authtoken

#Ask token
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 65432 &')

Get your authtoken from https://dashboard.ngrok.com/auth


KeyboardInterrupt: ignored

#Create Convolutional Neural Network
Use Keras Functional API to create a conv net with 1 input and 2 outputs

---

convolutional neural network from conv_net.py



In [ ]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.layers import *


class ConvNet:
    def __init__(self, input_shape, action_space):
        inputA = Input(shape=input_shape, name='inputA')
        inputB = Input(shape=input_shape, name='inputB')
        inputC = Input(shape=input_shape, name='inputC')

        x = Conv3D(32, 8, (4, 4, 4), activation='relu')(inputA)
        x = MaxPooling3D()(x)
        x = Conv3D(64, 1, (2, 2, 2), activation='relu')(x)
        x = MaxPooling3D()(x)
        x = Flatten()(x)
        x = models.Model(inputs=inputA, outputs=x)

        y = Conv3D(32, 8, (4, 4, 4), activation='relu')(inputB)
        y = MaxPooling3D()(y)
        y = Conv3D(64, 1, (2, 2, 2), activation='relu')(y)
        y = MaxPooling3D()(y)
        y = Flatten()(y)
        y = models.Model(inputs=inputB, outputs=y)

        z = Conv3D(32, 8, (4, 4, 4), activation='relu')(inputC)
        z = MaxPooling3D()(z)
        z = Conv3D(64, 1, (2, 2, 2), activation='relu')(z)
        z = MaxPooling3D()(z)
        z = Flatten()(z)
        z = models.Model(inputs=inputC, outputs=z)

        combined = Concatenate()([x.output, y.output, z.output])

        key_conv_output = layers.Dense(action_space[0])(combined)
        mouse_conv_output = layers.Dense(action_space[1])(combined)
        combined_input = [x.input, y.input, z.input]
        key_conv_net = models.Model(combined_input, key_conv_output, name="key_conv_net")
        mouse_conv_net = models.Model(combined_input, mouse_conv_output, name="mouse_conv_net")
        key_q_values = key_conv_net(combined_input)
        mouse_q_values = mouse_conv_net(combined_input)
        self.model = models.Model(inputs=combined_input, outputs=[key_q_values, mouse_q_values], name="full_conv_net")
        self.model.compile(loss="mean_squared_error",
                           optimizer=optimizers.RMSprop(lr=0.00025,
                                                        rho=0.95,
                                                        epsilon=0.01),
                           metrics=["accuracy"])

        self.model.summary()



#Set up DDQN Model
ddqn model.py from individual

In [ ]:
import os
import random
import numpy as np
import csv

# Hyper Parameters
StartingReplaySize = 500
ExplorationMax = 1
MemorySize = 900000
TrainingFrequency = 4
BatchSize = 32
Gamma = 0.99
ExplorationMin = 0.1
ExplorationSteps = 8500
ExplorationDecay = (ExplorationMax - ExplorationMin)/ExplorationSteps
ModelPersistenceUpdateFrequency = 10
TargetUpdateFrequency = 40


class ddqnModel:

    def __init__(self, input_shape, action_space, model_path):
        self.input_shape = input_shape
        self.action_space = action_space
        self.model_path = model_path
        self.ddqn = ConvNet(self.input_shape, action_space).model
        if os.path.isfile(self.model_path):
            self.ddqn.load_weights(self.model_path)

    def save_model(self):
        self.ddqn.save_weights(self.model_path)


class ddqnTrainer(ddqnModel):
    def __init__(self, input_shape, action_space):
        ddqnModel.__init__(self, input_shape, action_space, "gdrive/My Drive")
        self.ddqn_target = ConvNet(self.input_shape, action_space).model
        self.reset_target()
        self.epsilon = ExplorationMax
        self.memory = []

    def reset_target(self):
        self.ddqn_target.set_weights(self.ddqn.get_weights())

    def move(self, state):
        print(f"memory: {len(self.memory)}")
        if np.random.rand() < self.epsilon or len(self.memory) < StartingReplaySize:
            return random.randrange(self.action_space[0]), random.randrange(self.action_space[1])
        q_values = self.ddqn.predict(np.transpose(np.expand_dims(np.asarray(state).astype(np.float64), axis=0), [0, 2, 3, 1]), batch_size=1)
        return np.argmax(q_values[0]), np.argmax(q_values[1])

    def remember(self, current_state, action, reward, next_state):
        self.memory.append({"current_state": current_state,
                            "action": action,
                            "reward": reward,
                            "next_state": next_state})
        if len(self.memory) > MemorySize:
            self.memory.pop(0)

    def train(self):
        batch = np.asarray(random.sample(self.memory, BatchSize))
        if len(batch) < BatchSize:
            return
        current_states = []
        q_values = []
        max_q_values = []
        for entry in batch:
            current_state = np.transpose(np.expand_dims(np.asarray(entry["current_state"]).astype(np.float64), axis=0), [0, 2, 3, 1])
            current_states.append(current_state)
            next_state = np.expand_dims(np.asarray(entry["next_state"]).astype(np.float64), axis=0)
            next_state = np.transpose(next_state, [0, 2, 3, 1])
            # print(next_state.shape)
            next_state_prediction = self.ddqn_target.predict(next_state)
            # print(next_state_prediction)
            next_state_prediction = [next_state_prediction[0].ravel(), next_state_prediction[1].ravel()]
            next_q_value = [np.max(next_state_prediction[0]), np.max(next_state_prediction[1])]
            q = [self.ddqn.predict(current_state)[0][0], self.ddqn.predict(current_state)[1][0]]
            for i in range(len(entry["action"])):
                # print(len(entry["action"]))
                q[i][entry["action"][i]] = entry["reward"] + Gamma * next_q_value[i]
            q_values.append(q)
            max_q = [np.max(q[i]) for i in range(len(q))]
            max_q_values.append(max_q)
        max_q_values = np.asarray(max_q_values).flatten()
        q_values = np.asarray(q_values).T.tolist()
        # print(np.shape(q_values))
        # print(q_values[0])
        fitModel = self.ddqn.fit(np.asarray(current_states).squeeze(), y={
                                    "key_conv_net": np.asarray([q_value.tolist() for q_value in q_values[0]]).squeeze(),
                                    "mouse_conv_net": np.asarray([q_value.tolist() for q_value in q_values[1]]).squeeze()},
                                 batch_size=BatchSize, verbose=0)
        # print(fitModel.history)
        loss = fitModel.history["loss"][0]
        accuracy = sum([int(fitModel.history["key_conv_net_accuracy"][0]), int(fitModel.history["mouse_conv_net_accuracy"][0])])/2
        return loss, accuracy, sum(max_q_values.tolist())/len(max_q_values.tolist())

    def update_epsilon(self):
        self.epsilon -= ExplorationDecay
        self.epsilon = max(ExplorationMin, self.epsilon)

    def step_update(self, totalStep):
        if len(self.memory) < StartingReplaySize:
            return
        if totalStep % TrainingFrequency == 0:
            loss, accuracy, average_max_q = self.train()
            self.save_csv(path="loss.csv", score=loss)
            self.save_csv(path="accuracy.csv", score=accuracy)
            print(f"loss: {loss}, accuracy: {accuracy}, average_max_q: {average_max_q}")
        self.update_epsilon()
        if totalStep % ModelPersistenceUpdateFrequency == 0:
            self.save_model()
        if totalStep % TargetUpdateFrequency == 0:
            self.reset_target()
            print('{{"metric": "epsilon", "value": {}}}'.format(self.epsilon))
            print('{{"metric": "total_step", "value": {}}}'.format(totalStep))

    def save_csv(self, path, score):
        if not os.path.exists(path):
            with open(path, "w"):
                pass
        scores_file = open(path, "a")
        with scores_file:
            writer = csv.writer(scores_file)
            writer.writerow([score])

#Initiate main loop
Server uses sockets to listen for data

In [ ]:
import socket
from threading import *
import threading
from struct import *
from io import BytesIO
import zlib
import sys
import numpy as np


Frames = 28
InputShape = (Frames, 1080, 1920, 3)
TotalStepLimit = 5000000
ActionSpace = ("w", "a", "s", "d", "enter", "e")
MouseActionSpace = [[1061, 581], [1053, 619], [1031, 651], [999, 673], [961, 681], [922, 673], [890, 651], [868, 619], [861, 581], [868, 542], [890, 510], [922, 488], [961, 481], [999, 488], [1031, 510], [1053, 542]]

class Server:
    def __init__(self, host="", port=65432, input_shape=InputShape, learning_rate=0.01, party_link=""):
        self.host = host
        self.port = port
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        self.model = ddqnTrainer(InputShape, (len(ActionSpace), len(MouseActionSpace)) * 3)

    def handle_image(self, conn):
        try:
            conn.send(self.party_link)
            while True:
                length = unpack('>Q', conn.recv(8))[0]
                data = b''
                while len(data) < length:
                    to_read = length - len(data)
                    data += conn.recv(min(to_read, 4096))

                data = zlib.decompress(data)
                data = np.frombuffer(data)
                data = data.reshape(self.input_shape)
                self.stateBuff = np.append(self.buff, data).reshape(np.append(-1, self.input_shape))
                print(self.stateBuff.shape)

                assert len(b'\00') == 1
                conn.send(b'\00')
        except BrokenPipeError as e:
            self.sock.sendall(b'\01')
            print("Server shut down by user")
            sys.exit()
        except Exception as e:
            print(e)
            self.sock.close()
        finally:
            conn.close()

    def train():
        total_step = 0
        prev_score = 0
        prev_action = [0, 0]
        current_step = self.stateBuff
        print(total_step)
        while total_step <= TotalStepLimit:
            total_step += 1
            print(f"Step: {total_step}")
            action = self.model.move(self.stateBuff)
            sock.sendall(action)
            prev_action = action
            reward = np.sum(self.rewardBuff)
            self.model.remember(current_state, action, reward, next_state)
            thread = Thread(self.model.step_update, (total_step))
            thread.start()

    def start(self):
        try:
            self.sock.bind((self.host, self.port))
            self.sock.listen(5)
            while True:
                conn, addr = self.sock.accept()

                thread = Thread(target=self.handle_image, args=(conn,))
                thread.start()
                print(threading.active_count())
                if self.buff.shape[0] >= 3:
                    self.train()

        except KeyboardInterrupt as e:
            self.sock.sendall(b'\01')
            print("Server shut down by user")
            self.sock.close()
            sys.exit()
        finally:
            self.sock.close()


server = Server()
server.start()

Model: "full_conv_net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputA (InputLayer)             [(None, 28, 1080, 19 0                                            
__________________________________________________________________________________________________
inputB (InputLayer)             [(None, 28, 1080, 19 0                                            
__________________________________________________________________________________________________
inputC (InputLayer)             [(None, 28, 1080, 19 0                                            
__________________________________________________________________________________________________
key_conv_net (Model)            (None, 6)            97480230    inputA[0][0]                     
                                                                 inputB[0][0]         

BrokenPipeError: ignored

Kill Ngrok when Done

In [ ]:
# When done, kill Ngrok

!kill $(ps aux | grep './ngrok' | awk '{print $2}')